# Anyscale

[Anyscale](https://www.anyscale.com/) 是一个完全托管的 [Ray](https://www.ray.io/) 平台，您可以在其上构建、部署和管理可伸缩的 AI 和 Python 应用程序

这个例子介绍了如何使用 LangChain 与 `Anyscale` [service](https://docs.anyscale.com/productionize/services-v2/get-started) 交互 

In [ ]:
import os

os.environ["ANYSCALE_SERVICE_URL"] = ANYSCALE_SERVICE_URL
os.environ["ANYSCALE_SERVICE_ROUTE"] = ANYSCALE_SERVICE_ROUTE
os.environ["ANYSCALE_SERVICE_TOKEN"] = ANYSCALE_SERVICE_TOKEN

In [ ]:
from langchain.llms import Anyscale
from langchain import PromptTemplate, LLMChain

In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
llm = Anyscale()

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
question = "When was George Washington president?"

llm_chain.run(question)

使用 Ray，我们可以在不使用异步实现的情况下分发查询。这不仅适用于 Anyscale LLM 模型，还适用于任何其他未实现 `_acall` 或 `_agenerate` 的 Langchain LLM 模型

In [ ]:
prompt_list = [
    "When was George Washington president?",
    "Explain to me the difference between nuclear fission and fusion.",
    "Give me a list of 5 science fiction books I should read next.",
    "Explain the difference between Spark and Ray.",
    "Suggest some fun holiday ideas.",
    "Tell a joke.",
    "What is 2+2?",
    "Explain what is machine learning like I am five years old.",
    "Explain what is artifical intelligence.",
]

In [ ]:
import ray

@ray.remote
def send_query(llm, prompt):
    resp = llm(prompt)
    return resp

futures = [send_query.remote(llm, prompt) for prompt in prompt_list]
results = ray.get(futures)